<a id=aulas10e11></a>
# Aulas 10 e 11 - Expressões regulares

No começo do curso, vimos como é problemático procurar por certas substrings dentro de um texto. Na ocasião, queríamos descobrir se uma decisão judicial era pela procedência ou pela improcedência do pedido. Uma primeira tentativa, identificando se a substring "procedente" estava contida na string original, se mostrou insuficiente. Afinal, "procedente" está contido no texto "improcedente"! Como podemos garantir que nosso teste retornará verdadeiro quando "procedente" estiver no começo, mas não no final, de uma palavra? Note que não é suficiente procurar pela substring " procedente": pode ser que "procedente" seja a primeira palavra de uma linha, por exemplo.

Felizmente, as **expressões regulares** permitem que façamos buscas mais sofisticadas dentro de textos. O pacote `re` implementa essas funcionalidades no Python, permitindo que a nossa busca textual não seja simplesmente por matches exatos, mas possa incluir *padrões*. Para procurar por padrões, porém, temos que aprender uma sintaxe nova.

Esta sintaxe usa certos caracteres (em especial a contrabarra) para indicar o padrão que está sendo buscado. Por exemplo: "\b", dentro da sintaxe das expressões regulares, indica que o padrão buscado deve ser procurado apenas após os **limites de palavras**, que podem ser espaços em branco, mas também podem ser linhas em branco.

Sobre linhas, é importante notar que a contrabarra cumpre certas funções dentro das strings de Python que são diferentes daquelas cumpridas no caso das expressões regulares. Assim, é necessário especificar que não queremos que o Python interprete a nossa string normalmente, adicionando a letra r antes dela.

Fica mais fácil notar essa diferença com um exemplo:

In [1]:
print("Olha o que acontece quando fazemos \numa nova linha é gerada")

Olha o que acontece quando fazemos 
uma nova linha é gerada


In [2]:
print(r"Olha o que acontece quando fazemos \n : nada.")

Olha o que acontece quando fazemos \n : nada.


A forma como vamos trabalhar com expressões regulares em Python é a seguinte: nós vamos *compilar* alguns padrões e depois usar os métodos do objeto resultante (o padrão compilado) para fazer buscas nas strings maiores. Assim, se queremos buscar pelo padrão (exato) "procedente", o primeiro passo é criar um objeto compilado:

In [7]:
import re

procedente_exato = re.compile(r"procedente")

Podemos procurar pelo padrão em um determinado texto (por exemplo, o texto `decisao_improcedencia`, definido abaixo), chamando o método `.search()`.

In [8]:
decisao_improcedencia = """Sendo assim, em face das razões expostas, com fundamento nos 
poderes processuais outorgados ao Relator da causa (RTJ 139/53 – 
RTJ 168/174), e considerando, ainda, os precedentes firmados pelo 
Plenário desta Suprema Corte, julgo improcedente a presente ação de 
mandado de segurança, restando prejudicado, em consequência, o exame 
dos embargos de declaração opostos pelo impetrante."""

In [9]:
procedente_exato.search(decisao_improcedencia)

<re.Match object; span=(233, 243), match='procedente'>

Podemos usar alguns métodos específicos de objetos matchs, exemplo se queremos apenas o texto o ``.group()``.

Isso pode parecer pouco útil para essa busca simples, mas se estamos realizando busca por padrões mais sofisticados esse método terá usos.

In [17]:
print(procedente_exato.search(decisao_improcedencia).group())

procedente_exato.search(decisao_improcedencia).span()

procedente


(233, 243)

O método `.search()` retorna **a primeira substring** que combine ("match") o padrão buscado. Como pode ser observado na célula acima, o resultado contém tanto o span indicando os índices inicial (233) e final (243) da ocorrência, quanto o texto exato onde houve a combinação ("match"): "procedente".

Como já discutimos na primeira aula, o resultado dessa busca simples é insatisfatório. Afinal, a decisão citada é pela improcedência! Além do mais, não haveria necessidade de usar expressões regulares para fazer algo que poderia ser feito usando strings (com o operador `in` ou o método `find`, por exemplo).

A vantagem de usar expressões regulares está justamente no fato de que podemos estabelecer critérios mais sofisticados para que haja combinações. Podemos, por exemplo, estabelecer que só deve haver match com instâncias onde "procedente" inicie uma palavra nova, seja através de uma nova linha, de um espaço ou de um ponto final.

Podemos fazer isso usando uma expressão que tem um significado especial em expressões regulares: "\b".

In [19]:
procedente_palavra = re.compile(r"\bprocedente")
    
procedente_palavra.search(decisao_improcedencia)

O bloco acima não retorna nada porque não há nenhum match para a expressão regular "\bprocedente" no texto decisao_improcedencia. Em contraste, a expressão regular "\bimprocedente" deve retornar um match:

In [21]:
improcedente_palavra = re.compile(r"\bimprocedente")
    
improcedente_palavra.search(decisao_improcedencia)

<re.Match object; span=(231, 243), match='improcedente'>

Usar a expressão "\b", portanto, pode ser útil. Ela retorna matches somente caso o padrão buscado estiver imediatamente após um limite de palavras, como pontuação, nova linha ou espaços. Mas ainda podemos melhorar muito. O exemplo abaixo mostra como nossa busca ainda é sensível à variação na capitalização das letras.

In [22]:
decisao_improcedencia = """Sendo assim, em face das razões expostas, com fundamento nos 
poderes processuais outorgados ao Relator da causa (RTJ 139/53 – 
RTJ 168/174), e considerando, ainda, os precedentes firmados pelo 
Plenário desta Suprema Corte, julgo IMPROCEDENTE a presente ação de 
mandado de segurança, restando prejudicado, em consequência, o exame 
dos embargos de declaração opostos pelo impetrante."""

improcedente_palavra.search(decisao_improcedencia)

Felizmente, podemos adicionar o parâmetro `re.IGNORECASE` como um dos argumentos recebidos pela função `re.compile()`, o que faz com que o padrão seja agnóstico sobre maiúsculas ou minúsculas.

In [25]:
improcedente_palavra = re.compile(r"\bimprocedente", re.IGNORECASE)
    
improcedente_palavra.search(decisao_improcedencia)

<re.Match object; span=(231, 243), match='IMPROCEDENTE'>

Uma característica comum entre muitas das expressões regulares que vamos ver é que a expressão pode ser "negada" através do uso da letra como maiúscula. Assim, se "\b" procura pelo padrão apenas no começo de palavras, "\B" procura pelo padrão única e exclusivamente dentro de palavras maiores. Assim, a busca por "\Bprocedente" em decisao_improcedencia deve retornar um match.

In [27]:
procedente_meio_palavra = re.compile(r"\Bprocedente", re.IGNORECASE)
    
procedente_meio_palavra.search(decisao_improcedencia)

<re.Match object; span=(233, 243), match='PROCEDENTE'>

Uma lista não exaustiva de expressões regulares úteis em Python é a que se segue (adaptada [desta fonte](https://www.youtube.com/watch?v=K8L6KVGG-7o)):

.       - Qualquer caractere exceto nova linha <br>
\d      - Dígito (0-9)<br>
\D      - Qualquer coisa que não seja um dígito (0-9)<br>
\w      - "Word Character" (a-z, A-Z, 0-9, _)<br>
\W      - Qualquer coisa que não seja um "Word Character"<br>
\s      - Espaços em branco (espaço, tab, nova linha)<br>
\S      - Qualquer coisa que não seja um espaço em branco<br>
<br><br>
\b      - Limite de palavra<br>
\B      - Meio de palavra<br>
^       - Começo de string<br>
$       - Final de string<br>
<br>
[]      - Dá match em qualquer dos caracteres dentro dos colchetes<br>
[^ ]    - Dá match em qualquer caractere exceto os caracteres dentro dos colchetes
<br><br>
Quantificadores:<br>
\*       - 0 ou mais<br>
\+       - 1 ou mais<br>
\?       - 0 ou 1 vez<br>
{3}      - Número exato<br>
{3, 4}   - Range de números (mínimo, máximo)

Uma lista completa (em inglês) desses caracteres e da sintaxe de Regex pode ser encontrada na documentação do módulo: https://docs.python.org/3/library/re.html#regular-expression-syntax

Podemos usar as expressões regulares para acima encontrar as duas menções à Revista Trimestral de Jurisprudência do STJ em `decisao_improcedencia`. Antes de começar a construir o padrão, vamos olhar para o texto mais uma vez e encontrar as características que nos permitem identificar uma menção à RTJ.

In [28]:
decisao_improcedencia = """Sendo assim, em face das razões expostas, com fundamento nos 
poderes processuais outorgados ao Relator da causa (RTJ 139/53 – 
RTJ 168/174), e considerando, ainda, os precedentes firmados pelo 
Plenário desta Suprema Corte, julgo IMPROCEDENTE a presente ação de 
mandado de segurança, restando prejudicado, em consequência, o exame 
dos embargos de declaração opostos pelo impetrante."""

Sabemos que as menções que estamos procurando:

1) se iniciam com a substring "RTJ";

2) seguida por um espaço em branco;

3) seguida por um conjunto de 3 números;

4) seguido por uma barra;

5) seguida por um conjunto de 2 ou 3 números.

Começando com a parte fácil, podemos identificar menções à RTJ com o padrão "RTJ".

In [29]:
padrao_rtj = re.compile(r"RTJ")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 117), match='RTJ'>

Para adicionar o espaço em branco, temos algumas opções. Poderíamos, por exemplo, optar por dar match em qualquer caractere, usando a expressão regular ".".

In [30]:
padrao_rtj = re.compile(r"RTJ.")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 118), match='RTJ '>

Note que o nosso match tem um caractere a mais: o índice final agora é 118 e não 117. Podemos, porém, ser mais restritos.

Se sabemos que RTJ sempre se seguirá de um espaço em branco, podemos usar "\s" para dar match somente nesse tipo de caractere.

In [31]:
padrao_rtj = re.compile(r"RTJ\s")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 118), match='RTJ '>

O resultado é o mesmo, mas poderia ser diferente. Imagine que existisse uma revista chamada RTJRJ, a Revista do Tribunal de Justiça do Rio de Janeiro. Nossa primeira expressão regular daria um match com ela, enquanto a segunda, não:

In [32]:
padrao_rtj = re.compile(r"RTJ.")
padrao_rtj.search("RTJRJ")

<re.Match object; span=(0, 4), match='RTJR'>

In [33]:
padrao_rtj = re.compile(r"RTJ\s")
padrao_rtj.search("RTJRJ")

O próximo passo é encontrar uma sequência de três números. Podemos fazer isso adicionando 3 vezes a expressão "\d".

In [34]:
padrao_rtj = re.compile(r"RTJ\s\d\d\d")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 121), match='RTJ 139'>

Parece deselegante ter que repetir "\d" três vezes, portanto, podemos usar um quantificador exato e especificar que estamos buscando por três digitos usando {3}.

In [36]:
padrao_rtj = re.compile(r"RTJ\s\d{3}")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 121), match='RTJ 139'>

Na sequência, precisamos encontrar uma barra. Uma alternativa é usar a expressão ".", que dá match em qualquer caractere, mas já vimos que essa é uma estratégia preguiçosa. Especialmente porque temos uma expectativa exata do caractere utilizado.

Mais inteligente é usar a definição de "Word character", pra dar match apenas em coisas que não são "Word characters", ou seja, que não são letras, nem números. Afinal, pode ser que, ao invés de uma barra, o juiz resolva fazer a referência através de um traço ou de uma contrabarra.

In [37]:
padrao_rtj = re.compile(r"RTJ\s\d{3}\W")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 122), match='RTJ 139/'>

Finalmente, precisamos lidar com os últimos números. O problema é que nós não sabemos exatamente quantos digitos aparecerão nesse caso. Pode ser que a referência tenha três ou dois dígitos. Um quantificador útil para usar nesse caso é o `*`, que dará match em 0 ou mais ocorrências seguintes. Se tivermos certeza que haverá apenas um número, poderiamos também utilizar o quantirficador `+`.

In [38]:
padrao_rtj = re.compile(r"RTJ\s\d{3}\W\d*")
padrao_rtj.search(decisao_improcedencia)

<re.Match object; span=(114, 124), match='RTJ 139/53'>

Pronto! Conseguimos identificar a primeira referência feita à Revista Trimestral de Jurisprudência em `decisao_improcedencia`, mas sabemos que existe mais uma referência. O método adequado para encontrar todos os matches de um determinado padrão é `.findall()`.

In [20]:
padrao_rtj = re.compile(r"RTJ\s\d{3}\W\d*")
padrao_rtj.findall(decisao_improcedencia)

['RTJ 139/53', 'RTJ 168/174']

O resultado de `.findall()` é uma lista que contém cada um dos matches.

In [21]:
type(padrao_rtj.findall(decisao_improcedencia))

list

Outra forma de usar o findall é diretamente do módulo `re`, sem compilar a regex previamente.

In [23]:
re.findall(r"RTJ\s\d{3}\W\d*", decisao_improcedencia, re.IGNORECASE)

['RTJ 139/53', 'RTJ 168/174']

<a id=resumo></a>
# Resumo

Podemos utilizar expressões regulares e o módulo `re` para realizar operações mais complexas em strings, em especial o uso de regex nos possibilita encontrar padrões e não apenas matches exatos em textos.

* Utilizamos o módulo `re` para fazer operações com regex (expressões regulares) em Python.
* Para criar expressões utilizamos uma sintaxe própria com caracteres reservados que representam grupos de caracteres de texto.
* Podemos fazer algumas operações que fazíamos com string também com regex. Em nossa aula utilizamos busca, outro exemplo que podemos fazer é o uso do método `.replace()` para fazer substituições.


**Dica:** o site https://regex101.com/ cria explicações para ajudar a compreender e testar expressões regulares complexas, explicando cada parte da regex criada.

<a id=exercicio></a>
# Desafio

Na aula passada, vimos a estrutura do número do CNJ:

Outra coisa interessante que podemos fazer com strings é separá-las e extrair informações a partir disso. Um exemplo é a numeração única do CNJ. Segundo a [resolução 65/2008](http://www.cnj.jus.br///images/atos_normativos/resolucao/resolucao_65_16122008_04032013165912.pdf), os números do CNJ respondem ao seguinte padrão:

<br>
<center> NNNNNNN-DD.AAAA.J.TR.OOOO </center>
<br>

Onde N = o número do processo, DD = dígito verificador, AAAA = ano de protocolo, J = ramo da justiça, TR = tribunal e OOOO = órgão julgador.

Naquela ocasião, tivemos que desenvolver uma estratégia sofisticada para inferir as informações referentes a cada processo a partir dos índices de cada caracetere. Podemos usar expressões regulares para identificar menções a números do CNJ de maneira muito mais simples.

Guarde na variável `extrair_cnj` uma expressão regular compilada que extrai números do CNJ de um dado texto. Abaixo, carregamos novamente a sentença do TJRJ utilizada na aula passada para que você possa realizar testes.

In [43]:
import requests

#essa url faz o download do arquivo do google drive daquele id
google_url = "https://drive.google.com/uc?export=download&id="
file_id = "1JJ4Qf1gt1o2tTryhUFuCfloqFBJYkqdC"

tjrj_1_text = requests.get(google_url + file_id).content.decode(encoding = "latin9")

In [46]:
print(tjrj_1_text)

	Trata-se de reclamação através da qual a parte autora pleiteia a condenação da reclamada a cumprir uma obrigação de fazer e pagar indenização por danos morais. Resumidamente, alegou que reside no distrito de Porto Velho do Cunha e, já há algum tempo, o serviço de fornecimento de energia elétrica vem sendo prestado de forma deficiente. Disse que freqüentemente há interrupções. Solicitou reparos junto a reclamada. Houve até reuniões com os representantes da reclamada, mas as falhas permanecem. Ressaltou toda sorte de transtornos e findou requerendo a condenação da reclamada a normalizar, de uma vez por todas, o fornecimento de energia, bem como pagar indenização por danos morais. 

	A decisão de fls. 15 deferiu a antecipação da tutela. 

	Realizou-se AIJ, na qual a ré apresentou contestação escrita, acompanhada de documentos. 

	A questão consiste em determinar se os fatos constituem motivo para a imposição de indenização por danos morais. 

	Rápida pesquisa de jurisprudência das Turmas

In [ ]:
#seu código aqui

In [ ]:
#correcao
from correcoes import aula11_ex1
aula11_ex1(extrair_cnj)

# Precedentes

No STF (e também no STJ), raramente a menção a precedentes é feita através do número do CNJ. É muito mais comum que os ministros falem sobre uma classe processual e o número do processo (afinal, essa é a maneira canônica pela qual nos referimos a processos julgados pelo STF). Isso exige uma estratégia um pouco diferente para identificarmos menções a esses processos. Como exemplo, considere a ementa a seguir:

In [24]:
trecho_decisao = """
DECISÃO: Trata-se de “habeas corpus”, com pedido de medida liminar, 
impetrado contra decisão emanada de eminente Ministro de Tribunal 
Superior da União que, em sede de outra ação de “habeas corpus” ainda em 
curso no Superior Tribunal de Justiça (HC 390.905/SP), indeferiu medida 
liminar que lhe havia sido requerida em favor do ora paciente.

Sendo esse o contexto, passo a apreciar a admissibilidade, na espécie, 
da presente ação de “habeas corpus”. E, ao fazê-lo, devo observar que ambas 
as Turmas do Supremo Tribunal Federal firmaram orientação no sentido da 
incognoscibilidade desse remédio constitucional, quando impetrado, como 
no caso ora em análise, contra decisão monocrática proferida por Ministro 
de Tribunal Superior da União (HC 116.875/AC, Rel. Min. CÁRMEN 
LÚCIA – HC 117.346/SP, Rel. Min. CÁRMEN LÚCIA – HC 117.798/SP, 
Rel. Min. RICARDO LEWANDOWSKI – HC 118.189/MG, Rel. 
Min. RICARDO LEWANDOWSKI – HC 119.821/TO, Rel. Min. GILMAR 
MENDES – HC 121.684-AgR/SP, Rel. Min. TEORI ZAVASCKI – 
HC 122.381-AgR/SP, Rel. Min. DIAS TOFFOLI – HC 122.718/SP, Rel. 
Min. ROSA WEBER – RHC 114.737/RN, Rel. Min. CÁRMEN LÚCIA – 
RHC 114.961/SP, Rel. Min. DIAS TOFFOLI, v.g.)
"""

Como podemos extrair todos os HCs que são citados pela decisão? Sempre há menção à classe processual, seguida por um espaço, seguido pelo número do processo, com milhares separados por um ponto. Uma primeira estratégia, portanto, poderia ser a seguinte:

In [25]:
regex_hc = re.compile(r"HC\s\d*\W\d*")
regex_hc.findall(trecho_decisao)

['HC 390.905',
 'HC 116.875',
 'HC 117.346',
 'HC 117.798',
 'HC 118.189',
 'HC 119.821',
 'HC 121.684',
 'HC 122.381',
 'HC 122.718',
 'HC 114.737',
 'HC 114.961']

O problema com a expressão acima é que ela traz um RHC como sendo apenas um HC. Isso pode ser problemático. Uma maneira de lidar com isso é usando o operador lógico "ou", representado em expressões regulares como "|".

In [26]:
regex_hc_rhc = re.compile(r"\bHC\s\d*\W\d*|\bRHC\s\d*\W\d*")
regex_hc_rhc.findall(trecho_decisao)

['HC 390.905',
 'HC 116.875',
 'HC 117.346',
 'HC 117.798',
 'HC 118.189',
 'HC 119.821',
 'HC 121.684',
 'HC 122.381',
 'HC 122.718',
 'RHC 114.737',
 'RHC 114.961']

Ou

In [27]:
regex_hc_rhc_alt = re.compile(r"\bHC\s\d+\.\d+|\bRHC\s[0-9]+\.[0-9]+")
regex_hc_rhc_alt.findall(trecho_decisao)

['HC 390.905',
 'HC 116.875',
 'HC 117.346',
 'HC 117.798',
 'HC 118.189',
 'HC 119.821',
 'HC 121.684',
 'HC 122.381',
 'HC 122.718',
 'RHC 114.737',
 'RHC 114.961']

Podemos também utilizar o fato de a única alteração ser na classe, e a numeração ser a mesma.

Observa outra forma de fazermos essa identificação:

In [60]:
for match in regex_hc_rhc_alt.finditer(trecho_decisao):
    print(match.group())

HC 390.905
HC 116.875
HC 117.346
HC 117.798
HC 118.189
HC 119.821
HC 121.684
HC 122.381
HC 122.718
RHC 114.737
RHC 114.961


# Exercício extra

Guarde na variável `regex_extra` uma expressão regular compilada que seja capaz de extrair todas as menções a REs e AIs da decisão abaixo.

In [71]:
decisao = """
Decisão: O Supremo Tribunal Federal reconheceu a repercussão geral da controvérsia objeto dos presentes autos que versa sobre o direito, ou não, do segurado contribuinte da Previdência Social, à luz do art. 5, XXXVI, da Constituição Federal, ao cálculo da aposentadoria de acordo com a legislação vigente à época em que preenchidos os requisitos exigidos para a sua concessão, a qual se revela mais vantajosa do que aquela vigente à data da efetiva jubilação. Trata-se do Tema n. 334 da Gestão por Temas da Repercussão Geral do STF, que será submetido à apreciação do Pleno desta Corte, nos autos do RE n. 630.501/RS, Relatora a Ministra Ellen Gracie.

In casu, o acórdão recorrido assentou:

“APOSENTADORIA. DIREITO ADQUIRIDO. LEI N 7.787, DE 1989. INEXISTÊNCIA DE PREJUÍZO.

Inexiste direito adquirido do segurado de ver recalculada a renda mensal inicial de sua aposentadoria segundo as regras anteriores à Lei n 7.787, de 1989, sob as quais teria completado os requisitos do benefício, uma vez que a nova lei não trouxe nenhum prejuízo aos segurados” (fls. 86).

Destarte, aplicando a decisão Plenária no RE n. 579.431, secundada, a posteriori pelo AI n. 503.064-AgR-AgR, Relator o Ministro CELSO DE MELLO; AI n. 811.626-AgR-AgR, Relator o Ministro RICARDO LEWANDOWSKI, e RE n. 513.473-ED, Relator o Ministro CEZAR PELUSO, determino a devolução dos autos ao Tribunal de origem (artigo 328, parágrafo único, do RISTF c.c. artigo 543-B e seus parágrafos do Código de Processo Civil).

Publique-se.
"""

In [ ]:
#seu código aqui

In [ ]:
from correcoes import exercico_extra9
exercico_extra9(regex_extra)